# Import data

In [12]:
import numpy as np
import pandas as pd
import json

In [1]:
file = '../raw_data/games_dump.json'
with open(file) as data_file:    
    data = json.load(data_file) 

# Normalize the JSON file

In [7]:
#Normalize the complete dataframe and make a copy
df_normalized_teams = pd.json_normalize(data)
df_teams = df_normalized_teams.copy()

In [8]:
#Make a dataframe dedicated to the teams 
df_teams.drop(['teams.BLUE.players', 'teams.RED.players', 'picks_bans'], axis=1, inplace=True)
df_teams.tail(2)

,id,start,patch,winner,duration,video,teams.BLUE.name,teams.BLUE.total_turret_kills,teams.BLUE.total_inhibitor_kills,teams.BLUE.total_rift_herald_kills,teams.BLUE.total_dragon_kills,teams.BLUE.total_baron_kills,teams.RED.name,teams.RED.total_turret_kills,teams.RED.total_inhibitor_kills,teams.RED.total_rift_herald_kills,teams.RED.total_dragon_kills,teams.RED.total_baron_kills
18252,19524,2021-11-07T17:51:39+00:00,11.21,RED,2053.0,None,Inside Games,7.0,2.0,2.0,1.0,0.0,STOPWATCH eSports (Czech Team),9.0,1.0,0.0,4.0,2.0
18253,19525,2021-11-07T18:54:44+00:00,11.21,BLUE,2048.0,None,Inaequalis Academy,8.0,1.0,1.0,4.0,2.0,Dynamo Eclot,6.0,0.0,1.0,1.0,0.0


In [9]:
#Make a dataframe dedicated to the team BLUE
df_normalized_teams['teams.BLUE.players'].explode()
df_normalized_BLUE = pd.json_normalize(df_normalized_teams['teams.BLUE.players'].explode())
df_normalized_BLUE.sample(2)

,name,champion_name,champion_id,gd_15,kills_assists_15,deaths_15,gold_share_post_15,damage_share,jungle_proximity,support_proximity,...,total_monster_kills,total_assists,total_deaths,total_vision_score,total_damage_taken,total_damage_dealt,trueskill_sigma,trueskill_mu,win,side
34116,Adricesa,Hecarim,120,-30.0,5.0,0.0,NaN,0.150786,1.000000,0.307692,...,168.0,15.0,6.0,41.0,47759.0,163518.0,7.576075,26.538937,False,BLUE
88578,Wenbo (Marc Ostner),Tristana,18,1236.0,2.0,0.0,NaN,0.254190,0.128942,0.470315,...,37.0,2.0,1.0,64.0,12866.0,238414.0,4.893210,32.997210,True,BLUE


In [10]:
#Make a dataframe dedicated to the team RED
df_normalized_teams['teams.RED.players'].explode()
df_normalized_RED = pd.json_normalize(df_normalized_teams['teams.RED.players'].explode())
df_normalized_RED.sample(2)

,name,champion_name,champion_id,gd_15,kills_assists_15,deaths_15,gold_share_post_15,damage_share,jungle_proximity,support_proximity,...,total_monster_kills,total_assists,total_deaths,total_vision_score,total_damage_taken,total_damage_dealt,trueskill_sigma,trueskill_mu,win,side
46506,Nolja,Nautilus,111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,7.200323,33.486225,True,RED
60707,Seranok,Xerath,101,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,7.544152,23.805332,False,RED


##  Get the game id to properly analyze the data

We have two different dataframes for team Blue and team Red, later, we can merge them on the game id. This will also be useful when we want to come back for the y parameter! 

In [13]:
get_index = df_normalized_teams['id'].tolist()
index_preproc = np.asarray([[index] * 5 for index in get_index])
index_teams = index_preproc.reshape(len(df_normalized_teams) * 5).tolist()
df_normalized_RED['index_teams'] = index_teams
df_normalized_BLUE['index_teams'] = index_teams

# Retrieve the champions stats given a champion id

From the page https://ddragon.leagueoflegends.com/cdn/12.3.1/data/en_US/champion.json, get the stats of a champion given a key or id value.

In [14]:
champion_description = '../raw_data/lol_12_3_1.json'

with open(champion_description) as data_file:    
    data_champions = json.load(data_file)

In [15]:
# get the key and the stats for every champion in this version
# create a new dictionary with only the values of interest

champions_dict = {}
champions_names = list(data_champions['data'].keys())
for champion in champions_names:
    champions_dict[data_champions['data'][champion]['key']] = data_champions['data'][champion]['stats']
pd.DataFrame(champions_dict)

,266,103,84,166,12,32,34,1,523,22,...,777,83,350,154,238,221,115,26,142,143
hp,580.000,500.000,500.000,560.000,600.000,615.000,480.000,524.000,530.00,570.000,...,550.000,580.000,480.000,615.000,584.000,500.000,536.000,504.000,560.000,504.000
hpperlevel,90.000,82.000,105.000,90.000,106.000,75.000,82.000,88.000,88.00,87.000,...,85.000,100.000,70.000,95.000,85.000,85.000,92.000,82.000,92.000,79.000
mp,0.000,418.000,200.000,350.000,350.000,285.000,495.000,418.000,348.00,280.000,...,500.000,300.000,400.000,0.000,200.000,250.000,480.000,452.000,425.000,418.000
mpperlevel,0.000,25.000,0.000,40.000,40.000,40.000,45.000,25.000,42.00,32.000,...,0.000,60.000,45.000,0.000,0.000,45.000,23.500,50.000,25.000,25.000
movespeed,345.000,330.000,345.000,330.000,330.000,335.000,325.000,335.000,325.00,325.000,...,345.000,340.000,330.000,340.000,345.000,330.000,325.000,335.000,340.000,340.000
armor,38.000,18.000,23.000,26.000,44.000,30.000,21.000,19.000,28.00,26.000,...,28.000,39.000,25.000,33.000,32.000,23.000,22.000,24.000,21.000,29.000
armorperlevel,3.250,3.500,3.500,3.000,3.500,3.000,4.000,4.000,3.00,3.400,...,3.400,4.000,3.000,3.500,3.500,3.500,3.300,3.800,3.500,3.000
spellblock,32.000,30.000,37.000,30.000,32.000,32.000,30.000,30.000,30.00,30.000,...,32.000,32.000,25.000,32.000,32.000,30.000,30.000,30.000,30.000,30.000
spellblockperlevel,1.250,0.500,1.250,0.500,1.250,1.250,0.500,0.500,0.50,0.500,...,1.250,1.250,0.300,1.250,1.250,0.500,0.500,0.500,0.500,0.500
attackrange,175.000,550.000,125.000,500.000,125.000,125.000,600.000,625.000,550.00,600.000,...,175.000,175.000,500.000,175.000,125.000,500.000,550.000,550.000,550.000,575.000


# Features to drop (remove)

In [ ]:
#df_teams.drop(['video','teams.BLUE.name', 'teams.RED.name'], axis=1, inplace = True)